<a href="https://colab.research.google.com/github/mns0/GDP_GAN/blob/master/GDP_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from torchsummary import summary
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!nvidia-smi

Sat Mar 21 19:35:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import os
import random
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torchvision
import datetime
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np

from torch.nn import init


In [3]:
#get the data
dir = './drive/My Drive/gdp_data/'
#preprocessed in a previous 
train_data = 'Preprocessed_data_gdp_pc.dat'
gdp = pd.read_csv(dir+train_data)
#m = (gdp.columns != 'GDP') & (gdp.columns != 'Date') 
gdp = gdp.drop(['Unnamed: 0'], axis = 1).astype('float')
#gdp.loc[:,m]
#make even column dataframe
#gdp.drop(gdp[(gdp.shape[0]//15 * 15):].index, inplace=True)
#df_split = np.split(gdp.to_numpy(),gdp.shape[0]//15)
#df_split = np.asarray(df_split,dtype="float")
#df_split[0][0]
print(gdp.head())

for idx, c in enumerate(gdp.columns):
  l,m = gdp[c].min(), gdp[c].max()
  gdp[c] = 2*((gdp[c] - l)/(m-l))  - 1


print(gdp.head())


   CPILFESL  DFF  DTB3  DGS5  ...  PSAVERT   DSPI  GFDEGDQ188S      GDP
0      39.6  9.5  7.80  8.08  ...     11.8  732.1     30.08057  1.25734
1      39.6  9.5  7.80  8.08  ...     11.8  732.1     30.08057  1.25734
2      39.6  9.5  7.79  8.03  ...     11.8  732.1     30.08057  1.25734
3      39.6  9.0  7.82  8.08  ...     11.8  732.1     30.08057  1.25734
4      39.6  8.5  7.83  8.06  ...     11.8  732.1     30.08057  1.25734

[5 rows x 10 columns]
   CPILFESL       DFF      DTB3  ...  DSPI  GFDEGDQ188S       GDP
0      -1.0 -0.152330 -0.088578  ...  -1.0    -0.871205 -0.191709
1      -1.0 -0.152330 -0.088578  ...  -1.0    -0.871205 -0.191709
2      -1.0 -0.152330 -0.089744  ...  -1.0    -0.871205 -0.191709
3      -1.0 -0.197133 -0.086247  ...  -1.0    -0.871205 -0.191709
4      -1.0 -0.241935 -0.085082  ...  -1.0    -0.871205 -0.191709

[5 rows x 10 columns]


In [0]:
#Create a dataloader
class GDPData(Dataset):
  """GDP time series """
  def __init__(self, csv_file, series_length = 15, normalize = False):
    df = pd.read_csv(csv_file)
    df = df.drop(['Unnamed: 0'], axis = 1)
    if normalize:
      self.normalize_table = np.zeros((df.shape[1],2)) #(min, max)
      self.normalize(df)

    df.drop(df[(df.shape[0]//series_length * series_length):].index, inplace=True)
    x = df.loc[:, (df.columns != 'GDP') ]

    full = np.array(np.array_split(df.to_numpy(),df.shape[0]//series_length))
    x = np.array(np.array_split(x.to_numpy(),x.shape[0]//series_length))
    y = df["GDP"]
    y = np.array(np.array_split(y.to_numpy(),y.shape[0]//series_length))
    self.t = x.shape[0]
    #print("d", type(x),x.shape.shape)
    x = np.expand_dims(x,-1)
    full = np.expand_dims(full,-1)
    self.x = torch.from_numpy(x).float()
    self.y = torch.from_numpy(y).float()
    self.full = torch.from_numpy(full).float()


  def normalize(self,x):
    """
      Normalize data to [-1,1] range 
      for all predictors in dataframe
    """
    if not hasattr(self,'normalize_table'):
      raise Exception("Incorrectly calling normalize")
    for idx, c in enumerate(x.columns):
      l,m = x[c].min(), x[c].max()
      self.normalize_table[idx][0] = l
      self.normalize_table[idx][1] = m
      x[c] = 2*((x[c] - l)/(m-l))  - 1
    

  def __len__(self):
    return self.t

  def __getitem__(self,idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    xr  = self.x[idx]
    yr  = self.y[idx]
    #the conditional 
    full = self.full[idx]
    return xr, yr, full


In [0]:
import torch.nn as nn
import torch.nn.functional as F

class StatefulLSTM(nn.Module):
  def __init__(self,in_size,out_size):
    super(StatefulLSTM, self).__init__()
    self.lstm = nn.LSTMCell(in_size,out_size)
    self.out_size = out_size
    self.h, self.c = None, None
  
  def reset_state(self):
    self.h, self.c = None, None
  
  def forward(self,x):
    batch_size = x.data.size()[0]
    if self.h is None:
      state_size = [batch_size, self.out_size]
      self.c = Variable(torch.zeros(state_size)).cuda()
      self.h = Variable(torch.zeros(state_size)).cuda()
    self.h, self.c = self.lstm(x,(self.h,self.c))
    return self.h
    
class LockedDropout(nn.Module):
  def __init__(self):
    super(LockedDropout,self).__init__()
    self.m = None
  
  def reset_state(self):
    self.m = None
  
  def forward(self, x, dropout=0.5, train=True):
    if train==False:
      return x
    
    if (self.m is None):
      self.m = x.data.new(x.size()).bernoulli_(1 - dropout)
    mask = Variable(self.m, requires_grad=False) / (1 - dropout)
    return mask * x

In [0]:
# class Generator_RNN_test(nn.Module):
#   def __init__(self, predictor_dim, target_size=1, hidden_dim=1, num_layers=1, dropout_prob=0,train=True):
#     super(Generator_RNN_test, self).__init__()
#     self.train = train
#     self.dropout_prob = dropout_prob
#     self.lstm1 = StatefulLSTM(predictor_dim, hidden_dim)
#     self.bn_lstm1= nn.BatchNorm1d(hidden_dim)
#     self.dropout1 = LockedDropout() 

#     self.lstm2 = StatefulLSTM(hidden_dim, hidden_dim)
#     self.bn_lstm2= nn.BatchNorm1d(hidden_dim)
#     self.dropout2 = LockedDropout() 


#     self.fc_output = nn.Sequential(nn.Linear(hidden_dim*batch_size, target_size), nn.Tanh())

#   def reset_state(self):
#     self.lstm1.reset_state()
#     self.dropout1.reset_state()

#   def forward(self, input):
#     #self.reset_state()
#     # input - batch_size x time_steps x features
#     #self.reset_state()
#     batch_size, no_of_timesteps, features = input.size(0), input.size(1), input.size(2)
#     outputs = []

#     #for each timestep reduce the output
#     for i in range(no_of_timesteps):
#       #batch_size x features
#       h = self.lstm1(input[:,i,:])
#       h = self.bn_lstm1(h)
#       h = self.dropout1(h, dropout=self.dropout_prob ,train=self.train)
#       #batch_size x hidden_dim
#       h = self.lstm2(h)
#       h = self.bn_lstm2(h)
#       h = self.dropout2(h, dropout=self.dropout_prob ,train=self.train)
#       #batch_size x hidden_dim
#       h = self.fc_output(h)
#       #batch_size
#       print(h.size())

#       outputs.append(h)

#     outputs = torch.stack(outputs) # time_steps, batch_size, hidden_dim 
#     print("c", outputs.shape)
#     #outputs = torch.squeeze(output,dim=2)
#     #outputs = outputs.permute(1,2,0) # time_steps, features, batch_size

#     pool = nn.MaxPool1d(no_of_timesteps)

#     h = pool(outputs)
#     h = h.view(h.size(0),-1)
#     h = self.fc_output(h)
#     return h



class Generator_RNN(nn.Module):
  def __init__(self, predictor_dim, seq_len=15, target_size=1, hidden_dim=1, num_layers=1, dropout_prob=0,train=True):
    super(Generator_RNN, self).__init__()
    self.train = train
    self.dropout_prob = dropout_prob

    self.lstm1 = StatefulLSTM(predictor_dim, hidden_dim)
    self.bn_lstm1= nn.BatchNorm1d(hidden_dim)
    self.dropout1 = LockedDropout() 

    self.lstm2 = StatefulLSTM(hidden_dim, hidden_dim)
    self.bn_lstm2= nn.BatchNorm1d(hidden_dim)
    self.dropout2 = LockedDropout() 
    self.fc_output = nn.Sequential(nn.Linear(hidden_dim, target_size), nn.Tanh())

  def reset_state(self):
    self.lstm1.reset_state()
    self.dropout1.reset_state()

  def forward(self, inputx, inputc):
    #self.reset_state()
    # input - batch_size x time_steps x features
    #self.reset_state()
    input = torch.cat((inputx,inputc),2)
    batch_size, no_of_timesteps, features = input.size(0), input.size(1), input.size(2)
    outputs = []
    #print("In generator input size", input.shape)

    #lstm on each sequence 
    for i in range(batch_size):
      h = self.lstm1(input[i,:,:])
      h = self.bn_lstm1(h)
      h = self.dropout1(h)

      h = self.lstm2(h)
      h = self.bn_lstm2(h)
      h = self.dropout2(h)
      h = self.fc_output(h)
      h = torch.squeeze(h)
      outputs.append(h)    
    
    outputs = torch.stack(outputs) # batch_size, timesteps 
    return outputs


class Discriminator_RNN(nn.Module):
  def __init__(self, predictor_dim, target_size=1, hidden_dim=200, num_layers=1, dropout_prob=0,train=True):
    super(Discriminator_RNN, self).__init__()
    self.train = train
    self.dropout_prob = dropout_prob
    self.lstm1 = StatefulLSTM(predictor_dim, hidden_dim)
    self.bn_lstm1= nn.BatchNorm1d(hidden_dim)
    self.dropout1 = LockedDropout() 
    self.fc_output = nn.Sequential(nn.Linear(hidden_dim, target_size), nn.Sigmoid())

  def reset_state(self):
    self.lstm1.reset_state()
    self.dropout1.reset_state()

  def forward(self, _input):
    #self.reset_state()
    # input - batch_size x time_steps x features
    self.reset_state()
    no_of_timesteps = _input.shape[1]
    outputs = []
    for i in range(no_of_timesteps):
      h = self.lstm1(_input[:,i,:])
      h = self.bn_lstm1(h)
      h = self.dropout1(h,dropout=self.dropout_prob ,train=self.train)
      outputs.append(h)

    outputs = torch.stack(outputs) # time_steps, batch_size, features
    outputs = outputs.permute(1,2,0) # time_steps, features, batch_size

    pool = nn.MaxPool1d(no_of_timesteps)

    h = pool(outputs)
    h = h.view(h.size(0),-1)
    h = self.fc_output(h)
    return h

In [0]:
batch_size = 20
epochs = 200
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
gdp_dataset = GDPData(dir+train_data, normalize = True)
dataloader = torch.utils.data.DataLoader(gdp_dataset, batch_size=batch_size,
                                         shuffle=False)

kwagsD = {"target_size" : 1, "predictor_dim" : 10, "hidden_dim" : 200,\
         "num_layers" : 1, "dropout_prob" : 0, 'train' : True}

kwagsG = {"target_size" : 1, "predictor_dim" : 10, "hidden_dim" : 200,\
         "num_layers" : 1, "dropout_prob" : 0, 'train' : True}


netD = Discriminator_RNN(**kwagsD).to(device)
netG = Generator_RNN(**kwagsG).to(device)

criterion = nn.BCELoss().to(device)
optimizerD = optim.Adam(netD.parameters(), lr=0.001, 
                          betas=(0.9, 0.999), eps=1e-08, 
                          weight_decay=0, amsgrad=True)

optimizerG = optim.Adam(netG.parameters(), lr=0.001, 
                          betas=(0.9, 0.999), eps=1e-08, 
                          weight_decay=0, amsgrad=True)


# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
#fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1
fake_label = 0

#summary(netD,(20, 15, 10))

In [41]:
num_epochs = 100
for epoch in range(num_epochs):
  print("epoch: ", epoch)
  for i, data in enumerate(dataloader):
    #augment GDP data (x) with
    #other monatery data (conditional)
    #x + cond  (full)

    ############################
    # (1) Update D network: maximize log(D(x|y)) + log(1 - D(G(z|y)))
    ###########################


    conditional, x, full = data
    conditional, x = conditional.to(device), x.to(device)
    full = full.to(device)

    if full.size(0) < 20:
      continue


    batch_size, seq_len = full.size(0), full.size(1)
    netD.zero_grad()
    label = torch.full((batch_size,), real_label, device=device)
    output = netD(torch.squeeze(full,3)).view(-1)
    errD_real = criterion(output, label)
    errD_real.backward()
    D_x = output.mean().item()


    #training with the fake batch
    #G(Z|Y) normal noise + condition
    noise = torch.randn(batch_size, seq_len, 1, device=device)
    conditional = torch.squeeze(conditional,3)

    fake = netG(noise,conditional)
    label.fill_(fake_label)
    fake = torch.unsqueeze(fake,2)
    fake = torch.cat((fake,conditional),2)
    output = netD(fake.detach()).view(-1)

    errD_fake = criterion(output, label)
    errD_fake.backward()
    D_G_z1 = output.mean().item()
    errD = errD_real + errD_fake
    optimizerD.step()


    ############################
    # (2) Update G network: maximize log(D(G(z)))
    ###########################
    netG.zero_grad()
    label.fill_(real_label)  # fake labels are real for generator cost
    # Since we just updated D, perform another forward pass of all-fake batch through D
    output = netD(fake).view(-1)
    errG = criterion(output, label)
    errG.backward()
    D_G_z2 = output.mean().item()



    #additional condition
    noise = torch.randn(batch_size, seq_len, 1, device=device)



    # Update G
    optimizerG.step()

  if i % 1 == 0:
    print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
    % (epoch, num_epochs, i, len(dataloader),
       errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))






epoch:  0


RuntimeError: ignored